# 강화학습
- 에이전트: 인공지능 플레이어
- 환경: 에이전트가 솔루션을 찾기 위한 무대
- 행동: 에이전트가 환경 안에서 시행하는 상호작용
- 보상: 에이전트의 행동에 따른 점수 혹은 결과

## DQN의 주요 특징
- 기억하기 & 다시보기

In [3]:
import sys

!pip install gym
sys.path.append("c:/users/hwj43/anaconda3/lib/site-packages")

In [4]:
# gym : 카트풀 등 여러 게임 환경을 제공하는 패키지
# deque : 먼저 들어온 데이터가 먼저 나가게 되는 큐(queue) [FIFO]
#         deque는 double-ended queue의 약자로 큐와는 다르게 양쪽 끝에서 삽입과 삭제가 모두 가능
# random : 에이전트가 무작위로 행동할 확률을 구하기 위해 사용하는 파이썬의 기본 패키지
# math : 에이전트가 무작위로 행동할 확률을 구하기 위해 사용하는 파이썬의 기본 패키지
import gym
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import matplotlib.pyplot as plt

In [6]:
# hyper-parameter
EPISODES = 50      # 에피소드 반복 횟수 (총 플레이할 게임의 수)
EPS_START = 0.9    # 학습 시작 시 에이전트가 무작위로 행동할 확률 
EPS_END = 0.05     # 학습 막바지에 에이전트가 무작위로 행동할 확률
EPS_DECAY = 200    # 학습 진행 시 에이전트가 무작위로 행동할 확률을 감소시키는 값
GAMMA = 0.8        # 할인계수
LR = 0.001         # 학습률
BATCH_SIZE = 64    # 배치크기

'''
에피소드는 총 플레이 할 게임의 수  
EPS는 무작위로 행동할 확률로 모든 행동을 경험할 수 있도록 해줌(90%부터 시작해서 마지막엔 5%까지 떨어짐)
EPS_DECAY는 90에서 5로 떨어지는 감소율
GAMMA는 에이전트가 현재 보상을 미래 보상보다 얼마나 가치있게 여기는지이다.(할인계수의 개념)  
지금 받은 만원 말고, 1년 뒤에 받을 만원은 이자율만큼 곱해주어야함
'''

'\n에피소드는 총 플레이 할 게임의 수  \nEPS는 무작위로 행동할 확률로 모든 행동을 경험할 수 있도록 해줌(90%부터 시작해서 마지막엔 5%까지 떨어짐)\nEPS_DECAY는 90에서 5로 떨어지는 감소율\nGAMMA는 에이전트가 현재 보상을 미래 보상보다 얼마나 가치있게 여기는지이다.(할인계수의 개념)  \n지금 받은 만원 말고, 1년 뒤에 받을 만원은 이자율만큼 곱해주어야함\n'

### 입력
- 카트 위치
- 카트 속도
- 막대기 각도
- 막대기 속도

### 출력
- 0 (왼쪽)
- 1 (오른쪽)

### MEMORY

- 딥러닝 모델들은 보통 학습 데이터 샘플이 독립적이라 가정하지만, 강화학습에서는 연속된 상태가 강한 상관관계를 가지고 있음
    - 무작위로 가지고 오지않고 연속적인 경험을 학습하게 된다면 **초반의 몇가지 경험패턴**에만 치중해서 학습하게 됨
- 두번째는 신경망이 새로운 경험을 전 경험에 겹쳐 쓰며 쉽게 잊어버림
### "기억하기" 기능 추가  

  
- **이전 경험들을 배열에 담아 계속 재학습**시키면 신경망이 잊지 않게 한다는 아이디어
- 기억한 경험들은 학습을 할 때 무작위로 뽑아 경험간의 상관관계를 줄인다  

- 각 경험은 상태, 행동, 보상등을 담아야 함
    - 이전 경험들에 관한 기억을 담고자 **memory라는 배열**을 만든다
    
```python
self.memory = [(상태, 행동, 보상, 다음상태)...]
```

복잡한 모델을 만들때는 memory를 클래스로 구현하기도 하지만, 이번예제에서는 사용하기 가장 간단한 큐(queue) 자료구조 사용  
파이썬에서의 deque의 maxlen을 지정해주면 큐가 가득 찼을때 제일 오래된 요소부터 없애줌

In [4]:
class DQNAgent:
    def __init__(self):
        # 4개의 입력, 2개의 출력
        self.model = nn.Sequential(
            nn.Linear(4, 256),
            nn.ReLU(),
            nn.Linear(256,2))
        
        #optimizer
        self.optimizer = optim.Adam(self.model.parameters(), LR)
        
        #학습을 반복할 때마다 증가하는 변수
        self.steps_done = 0

        self.memory = deque(maxlen = 10000)
   
    #self.memory 배열에 새로운 경험을 덧붙일 memorize() 함수를 만듦
    #memorize() 함수는 self.memory 배열에 현재상태(state), 현재 상태에서 한 행동(action), 행동에 대한 보상(reward), 행동으로 인해 생성된 상태(next_state)
    def memorize(self, state, action, reward, next_state):
        self.memory.append((state, action,
                            torch.FloatTensor([reward]),
                           torch.FloatTensor([next_state])))
        
        
    #앱실론의 값이 크면 신경망 학습하여 행동하는 쪽으로, 낮으면 무작위로 행동
    #이 알고리즘을 epsilon-greedy 알고리즘이라고함
    def act(self, state):
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * self.steps_done / EPS_DECAY)
        self.steps_done += 1 #학습 진행 될때마다 +1
        if random.random() > eps_threshold:
            return self.model(state).data.max(1)[1].view(1,1)
        else:
            return torch.LongTensor([[random.randrange(2)]])
    
    #경험으로부터 배우기
    #에이전트가 기억하고 다시 상기하는 과정(experience replay)
    #self.memory에 저장된 경험들의 수가 아직 배치 크기(BATCH_SIZE) 보다 커질 때까진 return으로 학습을 거르고, 만약 경험이 충분히 쌓이면 self.memory 큐에서 무작위로 배치 크기만큼의 '경험'들 가지고 오기
    # 경험들을 무작위로 가지고오면 각 경험 샘플간의 상관성 줄이기 가능
    
    def learn(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        states, actions, rewards, next_states = zip(*batch)
        
        states = torch.cat(states)
        actions = torch.cat(actions)
        rewards = torch.cat(rewards)
        next_states = torch.cat(next_states)
        
        current_q = self.model(states).gather(1, actions)
        max_next_q = self.model(next_states).detach().max(1)[0]
        expected_q = rewards + (GAMMA * max_next_q)
        
        loss = F.mse_loss(current_q.squeeze(), expected_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [5]:
#학습 준비하기
env = gym.make('CartPole-v0') # 환경 만들어주기
agent = DQNAgent() #에이전트
score_history = []  #점수 저장

In [6]:
# 학습 시작
for e in range(1, EPISODES +1): #얼마나 많은 게임을 진행하느냐
    state = env.reset() #게임을 시작할때마다 cartpole 게임환경의 상태를 초기화
    steps = 0  
    
    while True: 
        env.render() #게임 화면을 띄움
        
        state = torch.FloatTensor([state]) #현재 게임의 상태 state를 텐서로 만듦
        action = agent.act(state) #에이전트의 행동함수 act()의 입력으로 사용
        
        next_state, reward, done, _  = env.step(action.item()) 
        #action 변수는 파이토치 텐서. 
        #item()함수로 에이전트가 한 행동의 번호를 추출하여 step()함수에 입력해주면 에이전트의 행동에 따른 다음 상태(next_state), 보상(reward), 그리고 종료여부(done) 출력
        
        
        #게임이 끝났을 경우 마이너스 보상 주기
        if done:
            reward = -1 # 막대가 넘어져서 게임이 끝났을 경우
        agent.memorize(state, action, reward, next_state) # 이 경험을 기억
        agent.learn()
        
        state = next_state
        steps += 1
        
        
        # 게임이 끝나면 done이 True 가 된다
        if done:
            print("에피소드:{} 점수:{}".format(e, steps))
            score_history.append(steps)
            break

에피소드:1 점수:28
에피소드:2 점수:16
에피소드:3 점수:10
에피소드:4 점수:9
에피소드:5 점수:14
에피소드:6 점수:11
에피소드:7 점수:13
에피소드:8 점수:13
에피소드:9 점수:11
에피소드:10 점수:12
에피소드:11 점수:13
에피소드:12 점수:15
에피소드:13 점수:11
에피소드:14 점수:10
에피소드:15 점수:13
에피소드:16 점수:17
에피소드:17 점수:14
에피소드:18 점수:18
에피소드:19 점수:9
에피소드:20 점수:11
에피소드:21 점수:10
에피소드:22 점수:12
에피소드:23 점수:12
에피소드:24 점수:10
에피소드:25 점수:14
에피소드:26 점수:13
에피소드:27 점수:15
에피소드:28 점수:12
에피소드:29 점수:30
에피소드:30 점수:14
에피소드:31 점수:18
에피소드:32 점수:56
에피소드:33 점수:43
에피소드:34 점수:45
에피소드:35 점수:48
에피소드:36 점수:101
에피소드:37 점수:200
에피소드:38 점수:150
에피소드:39 점수:88
에피소드:40 점수:170
에피소드:41 점수:190
에피소드:42 점수:165
에피소드:43 점수:176
에피소드:44 점수:200
에피소드:45 점수:185
에피소드:46 점수:200
에피소드:47 점수:200
에피소드:48 점수:200
에피소드:49 점수:200
에피소드:50 점수:200


In [ ]:
# 점수 기록을 그래프로 그려서 시각화
plt.plot(score_history)
plt.ylabel('score')
plt.show()